In [1]:
cd /content/drive/MyDrive/Projects/Llama2_Garden/

[Errno 2] No such file or directory: '/content/drive/MyDrive/Projects/Llama2_Garden/'
/content


In [2]:
!source venv/bin/activate

/bin/bash: line 1: venv/bin/activate: No such file or directory


In [3]:
!pip install chromadb
!pip install -q trl==0.4.7 transformers==4.36.1

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.0/509.0 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 24.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.0/92.0 kB 13.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.5/60.5 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.6/40.6 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.4/5.4 MB 68.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.4/6.4 MB 60.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.9/57.9 kB 9.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.6/105.6 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 9.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 698.9/698.9 kB 6

In [4]:
import chromadb
import os
import torch
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
    BitsAndBytesConfig,
    HfArgumentParser,
    TrainingArguments,
    pipeline,
    logging,
    Conversation
)

#Vector Database Setup

In [5]:
chroma_client = chromadb.PersistentClient(path="gardening_database/")
collection = chroma_client.get_or_create_collection(name = "my_collection")

In [6]:
# df = pd.read_csv("gardening_dataset.csv")
# for i,item in enumerate(df["text"].to_list()):
#   collection.add(
#       documents=[item],
#       ids=[str(i)],
#       metadatas=[{"content":item}]
#   )
#   print(f"Created vector {i}")

#Setting up Llama-2 LLM

In [7]:
model_name = "NousResearch/Llama-2-7b-chat-hf"
device_map = {"": 0}

In [8]:
base_model = AutoModelForCausalLM.from_pretrained(
    model_name,
    low_cpu_mem_usage=True,
    return_dict=True,
    torch_dtype=torch.float16,
    device_map=device_map,
)
tokenizer = AutoTokenizer.from_pretrained(model_name, trust_remote_code=True)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = "right"

config.json:   0%|          | 0.00/583 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

...

tokenizer_config.json:   0%|          | 0.00/746 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/21.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/435 [00:00<?, ?B/s]

#Setting up Chatbot

In [22]:
system_content = """You are a gardening assistant bot. Your primary task is to answer questions based STRICTLY on gardening.

      RULES:
      - ONLY answer if the question relates directly to the provided context.
      - Do NOT provide information that is not explicitly mentioned in the context. Avoid speculating or adding details from outside the context.
      - If the question is not based on gardening, STRICTLY respond with  'I don't know'.
      - If the question does NOT directly match with the context, respond with  'I don't know'.
      - If no context is provided, always respond with 'I don't know'.
      - Ensure the answers are relevant and helpful in the context of gardening. However, ensure the elaboration is strictly based on the context.
      - If the question is a greeting (like "Hello", "Hi", etc.), respond with a corresponding greeting.

      NOTE: Stick to the context. If uncertain, respond with 'I don't know'."""

conversation = Conversation()

      # - If the question is not based on gardening, STRICTLY respond with  'I don't know'.
      # - If the question does NOT directly match with the context, respond with  'I don't know'.

In [29]:
del pipe
torch.cuda.empty_cache()
pipe = pipeline(task="conversational", model=base_model, tokenizer=tokenizer, max_new_tokens=750)
def get_reply(prompt, conversation):
  context = ""
  replys = collection.query(
      query_texts = [prompt],
      n_results = 4
  )
  for dist,doc in zip(replys["distances"][0],replys["documents"][0]):
    if dist <= 0.8:
      context += doc
      c+=1
  conversation.add_message(message = {
      "role": "system",
      "content": f"""You are a gardening assistant bot. Your primary task is to answer questions based STRICTLY on gardening.

      RULES:
      - ONLY answer if the question relates directly to the provided context.
      - Do NOT provide information that is not explicitly mentioned in the context. Avoid speculating or adding details from outside the context.
      - If no context is provided, always respond with 'I don't know'.
      - Ensure the answers are relevant and helpful in the context of gardening. However, ensure the elaboration is strictly based on the context.
      - If the question is a greeting (like "Hello", "Hi", etc.), respond with a corresponding greeting.

      NOTE: Stick to the context. If uncertain, respond with 'I don't know'.

      context: {context}"""
  })
  conversation.add_message(message = {
      "role": "user",
      "content": prompt
  })
  result=""
  try :
    conversation = pipe(conversation)
    result = conversation.messages[-1]["content"]
  except:
    conversation = Conversation()
    result = get_reply(prompt, conversation)
  finally:
    return result

In [30]:

while True:
  prompt = input("User: ")
  if prompt == "exit":
    print("Thank you for gardening with us")
    break
  res = get_reply(prompt, conversation)
  print(f"Assistant: ",res,"\n\n")

User: how to climb a tree
Assistant:   Great, I'm happy to help with your gardening question! Climbing a tree can be a fun and rewarding experience, but it's important to do it safely and with proper technique. Here are some steps you can follow to climb a tree:

1. Choose the right tree: Look for a tree with a sturdy trunk and branches that are easy to grip. Avoid trees with weak or rotting branches, as they can break and cause injury.
2. Warm up and stretch: Before you start climbing, do some light stretching and warm-up exercises to loosen up your muscles. This will help you climb more efficiently and reduce the risk of injury.
3. Find a good starting point: Look for a sturdy branch that is about shoulder height and has good footing. This will be your starting point for the climb.
4. Use proper footing: Place your feet on the branch, with your knees bent and your weight evenly distributed between both feet. Make sure your feet are secure and can grip the branch.
5. Use your arms to 

In [26]:
questions =[
    "how to grow potatoes",
    "how to grow basil",
    "How to play cricket"
]
for q in questions:
  print("User: ", q)
  res = get_reply(q, conversation)
  print("Assistant: ", res, "\n\n")

User:  how to grow potatoes
Assistant:   Great! I'm here to help you with your gardening questions. To grow potatoes, you will need to follow these steps:

1. Choose a sunny location with well-draining soil. Potatoes prefer a slightly acidic soil with a pH between 4.5 and 7.0.
2. Obtain seed potatoes, which are small, whole potatoes or pieces of potatoes that have at least one "eye" each. You can purchase seed potatoes from a garden center or nursery, or use leftover potatoes from the supermarket that have not been treated with a sprout inhibitor.
3. Chit the seed potatoes by placing them in a light, cool place with the "eyes" facing upwards. This will help the potatoes to sprout before planting.
4. Plant the seed potatoes in late spring or early summer, when the soil is warm and the danger of frost has passed. Dig a trench about 8 inches deep and 12 inches wide, and place the seed potatoes in the trench with the "eyes" facing upwards. Cover the seed potatoes with soil and gently firm 

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/base.py:1101: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(


Assistant:   I don't know. 


